In [1]:
import sys 
sys.path.append('../scripts/')
from kf import *   #誤差楕円を描くのに利用

In [2]:
def draw(xs, ms, zlist): 
    ##世界の描画##
    fig = plt.figure(figsize=(4,4))
    ax = fig.add_subplot(111) 
    ax.set_aspect('equal')
    ax.set_xlim(-5,5)
    ax.set_ylim(-5,5) 
    ax.set_xlabel("X",fontsize=10) 
    ax.set_ylabel("Y",fontsize=10)  
    
    ##軌跡の描画##
    poses = [xs[s] for s in range(len(xs))]
    ax.scatter([e[0] for e in poses], [e[1] for e in poses], s=5, marker=".", color="black")
    ax.plot([e[0] for e in poses], [e[1] for e in poses], linewidth=0.5, color="black")

    ##センサ値の描画##
    for t in range(len(xs)):
        if t not in zlist: continue
            
        for obs in zlist[t]:
            x, y, theta = xs[t]
            ell, phi = obs[1]
            mx = x + ell*math.cos(theta + phi)
            my = y + ell*math.sin(theta + phi)
            ax.plot([x,mx], [y,my], color="pink", alpha=0.5)
        
    ##ランドマークの推定位置の描画## 
    ax.scatter([e[0] for e in ms], [e[1] for e in ms], s=10, marker="*", color="blue")
    print(ms)
    
    
    ##描画実行##
    plt.show()

In [3]:
###データの読み込み###
delta = 0.0
xs = {}     #軌跡のデータ（ステップ数をキーにして姿勢を保存）
zlist = {} #センサ値のデータ（ステップ数をキーにして、さらにその中にランドマークのIDとセンサ値をタプルで保存）
us = {}    #制御入力のデータ（ステップ数をキーにして保存）

with open("log.txt") as f:
    for line in f.readlines():
        tmp = line.rstrip().split()
        
        step = int(tmp[1])
        if tmp[0] == "delta":
            delta = float(tmp[1])
        elif tmp[0] == "x": #姿勢のレコードの場合
            xs[step] = np.array([float(tmp[2]), float(tmp[3]), float(tmp[4])]).T
        elif tmp[0] == "z": #センサ値のレコードの場合
            if step not in zlist:  #まだ辞書が空の時は空の辞書を作る
                zlist[step] = []
            zlist[step].append((int(tmp[2]), np.array([float(tmp[3]), float(tmp[4])]).T))
        elif tmp[0] == "u": #制御入力の場合
            us[step] = np.array([float(tmp[2]), float(tmp[3])]).T

In [4]:
##空の精度行列と一次の項のベクトルの準備## 
t_end = len(xs) - 1 #step数（数式中のT）

##観測されたランドマークのリスト（精度行列に設けるもの）のセット##
observed_landmarks = set()
for t in range(len(xs)):
    if t not in zlist: continue

    for obs in zlist[t]:
        observed_landmarks.add(obs[0])

##精度行列の次元を求めて精度行列と一次の項のベクトルの初期化##
dim = (t_end + 1)*3 + len(observed_landmarks)*2
Omega = np.zeros((dim, dim))
xi = np.zeros((dim, 1))

##確認##
print(t_end)                           #終了時刻
print(observed_landmarks) #観測されたランドマークのID
print(Omega)                        #精度行列
print(xi)                                  #一次の項のベクトル

31
{0, 1, 2, 3, 4, 5}
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


In [5]:
mxs, mys = {}, {} #ランドマークごとにx座標を貯める辞書
for j in observed_landmarks: mxs[j], mys[j] = [], [] #ランドマークごとに座標のリストを初期化

for t in range(len(xs)):
    if t not in zlist: continue  #センサ値のないステップは飛ばす
            
    for obs in zlist[t]:             #センサ値ごとに地図上の位置を計算
        x, y, theta = xs[t]         #デッドレコニング上の座標
        ell, phi = obs[1]            #センサ値
        j = obs[0]                      #ランドマークのID
        mx = x + ell*math.cos(theta + phi)    #ランドマークの座標
        my = y + ell*math.sin(theta + phi)    #ランドマークの座標
        
        mxs[j].append(mx)     #計算した座標をリストに加える
        mys[j].append(my)
        
mx_avg, my_avg = {}, {}
hat_ms = []
for j in observed_landmarks:  #ランドマークごとに平均値を出す
    mx_avg[j] = sum(mxs[j])/len(mxs[j])
    my_avg[j] = sum(mys[j])/len(mys[j])
    hat_ms.append(np.array([mx_avg[j], my_avg[j]]).T)
    
hat_xs = xs  #姿勢の初期値はxsをそのまま使う
hat_ms = hat_ms

print(hat_ms)

[array([-2.13800045,  3.38554522]), array([-1.51314626, -4.10141355]), array([ 3.46510084, -0.17765306]), array([1.91972816, 2.59158263]), array([ 0.72851143, -0.58363871]), array([-3.62358709,  0.93716617])]


In [6]:
##Ω_0を作る## ###graphslam3omegazero
alpha = 0.001
Sigma_0 = np.diag([0.001**2, 0.001**2, 0.001**2])

Omega_0 = np.zeros((dim, dim))                        #ゼロ行列で初期化
Omega_0[0:3, 0:3] += np.linalg.inv(Sigma_0) #左上の角に3x3の精度行列を足す

Omega_0

array([[1000000.,       0.,       0., ...,       0.,       0.,       0.],
       [      0., 1000000.,       0., ...,       0.,       0.,       0.],
       [      0.,       0., 1000000., ...,       0.,       0.,       0.],
       ...,
       [      0.,       0.,       0., ...,       0.,       0.,       0.],
       [      0.,       0.,       0., ...,       0.,       0.,       0.],
       [      0.,       0.,       0., ...,       0.,       0.,       0.]])

In [7]:
##Ω_x,xを作る##
motion_noise_stds = {"nn":0.19, "no":0.001, "on":0.13, "oo":0.2}

def R(t): #R_tを返す
    theta = xs[t][2]
    nu, omega = us[t]
    A = delta * np.array([[math.cos(theta), 0.0], [math.sin(theta), 0.0], [0.0, 1.0]])
    M = np.diag(
        [motion_noise_stds["nn"]**2*abs(nu)/delta +motion_noise_stds["no"]**2*abs(omega)/delta,
             motion_noise_stds["on"]**2*abs(nu)/delta + motion_noise_stds["oo"]**2*abs(omega)/delta]
    )
    print(A)
    return A.dot(M).dot(A.T)

def F(t): #F_{t-1}を返す（添字注意）
    F = np.array(np.eye(3))
    x, y, theta = xs[t-1]
    nu, omega = us[t]
    
    if abs(omega) < 10e-5:
        F[0, 2] = - nu * delta * math.sin(theta)
        F[1, 2] = nu * delta * math.cos(theta)
    else:
        F[0, 2] = nu / omega * (math.cos(t + omega * delta) - math.cos(theta))
        F[1, 2] = nu / omega * (math.sin(t + omega * delta) - math.sin(theta))
        
    return F

##必要な行列を先に計算##
Rs, Fs, Rinvs = {}, {}, {}
for t in range(1, t_end+1):
    Rs[t] = R(t)
    print(Rs[t])
    Rinvs[t] = np.linalg.inv(Rs[t])
    Fs[t-1] = F(t)
    
##各x_{t-1}, x_tペアの精度行列を作る
Omega_xx = []          ###x_{t-1}とx_tに対する精度行列を保管するリスト
for t in range(1, t_end+1):
    Omega_tmp = np.zeros((dim, dim)) 
    Omega_tmp[3*t:3*t+3, 3*t:3*t+3] += Fs[t-1].T.dot(Rinvs[t]).Fs[t-1]

[[2.89777748 0.        ]
 [0.77645714 0.        ]
 [0.         3.        ]]
[[0.0202093  0.00541507 0.        ]
 [0.00541507 0.00145096 0.        ]
 [0.         0.         0.02061198]]
[[2.59807621 0.        ]
 [1.5        0.        ]
 [0.         3.        ]]
[[0.0162452  0.00937917 0.        ]
 [0.00937917 0.00541507 0.        ]
 [0.         0.         0.02061198]]


LinAlgError: Singular matrix

In [ ]:
draw(hat_xs, hat_ms, zlist)

In [ ]:
Rs

In [ ]:
Fs

In [8]:
A = np.array([[0.0202093,  0.00541507, 0.        ],
 [0.00541507, 0.00145096, 0.        ],
 [0. ,        0. ,        0.02061198]])

In [10]:
np.linalg.inv(A)

array([[-1.49311205e+07,  5.57238397e+07,  0.00000000e+00],
       [ 5.57238397e+07, -2.07964033e+08, -0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  4.85154750e+01]])